In [ ]:
!pip install opencv-python opencv-contrib-python numpy pandas keras tensorflow

In [21]:
import cv2
import numpy as np
from keras.models import load_model
from datetime import datetime
import pandas as pd
import os
from time import sleep

# Load Emotion Detection Model
emotion_model = load_model("fer2013_mini_XCEPTION.102-0.66.hdf5", compile=False)
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Load Face Detection and Recognition Models
face_detector = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

# --------------------- Register & Train ---------------------
def capture_samples(student_name, num_samples=5):
    os.makedirs(f"student_images/{student_name}", exist_ok=True)
    cap = cv2.VideoCapture(0)
    count = 0
    while count < num_samples:
        ret, frame = cap.read()
        if not ret:
            continue
        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
                                     (300, 300), (104.0, 177.0, 123.0))
        face_detector.setInput(blob)
        detections = face_detector.forward()

        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.7:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x1, y1, x2, y2) = box.astype("int")
                face = frame[y1:y2, x1:x2]
                if face.size > 0:
                    gray_face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                    cv2.imwrite(f"student_images/{student_name}/sample_{count}.jpg", gray_face)
                    count += 1
                    sleep(1)
        cv2.imshow("Capturing Samples", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    cap.release()
    cv2.destroyAllWindows()

def train_recognizer():
    faces, ids = [], []
    label_ids = {}
    current_id = 0

    for root, dirs, files in os.walk("student_images"):
        for file in files:
            if file.endswith(("jpg", "png")):
                path = os.path.join(root, file)
                label = os.path.basename(root)
                if label not in label_ids:
                    label_ids[label] = current_id
                    current_id += 1
                img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    faces.append(img)
                    ids.append(label_ids[label])

    if faces:
        face_recognizer.train(faces, np.array(ids))
    return label_ids

label_ids = train_recognizer()
id_to_name = {v: k for k, v in label_ids.items()}

# --------------------- Emotion Detection ---------------------
def detect_emotion(face_img):
    try:
        face_img = cv2.resize(face_img, (64, 64))
        face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
        face_img = np.reshape(face_img, [1, 64, 64, 1]).astype('float32') / 255.0
        prediction = emotion_model.predict(face_img)
        return emotion_labels[np.argmax(prediction)]
    except:
        return "Unknown"

# --------------------- Take Attendance ---------------------
def take_attendance():
    attendance = {name: {'status': 'Absent', 'emotion': 'Unknown', 'time': ''} for name in id_to_name.values()}
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
                                     (300, 300), (104.0, 177.0, 123.0))
        face_detector.setInput(blob)
        detections = face_detector.forward()

        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.7:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x1, y1, x2, y2) = box.astype("int")
                face = frame[y1:y2, x1:x2]
                if face.size > 0:
                    emotion = detect_emotion(face)
                    gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                    try:
                        id_, conf = face_recognizer.predict(gray)
                        if conf < 100:
                            name = id_to_name[id_]
                            attendance[name] = {
                                'status': 'Present',
                                'emotion': emotion,
                                'time': datetime.now().strftime("%H:%M:%S")
                            }
                            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                            cv2.putText(frame, f"{name} - {emotion}", (x1, y1 - 10),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                    except:
                        pass

        cv2.imshow("Attendance System", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Save to CSV
    df = pd.DataFrame.from_dict(attendance, orient='index').reset_index()
    df.columns = ['Student', 'Status', 'Emotion', 'Time']
    df['Date'] = datetime.now().strftime("%Y-%m-%d")
    os.makedirs("attendance_reports", exist_ok=True)
    df.to_csv(f'attendance_reports/{df["Date"][0]}.csv', index=False)
    print(f"✅ Attendance saved to attendance_reports/{df['Date'][0]}.csv")

# --------------------- Usage ---------------------
# 👉 Register a student:
#register_student("student_name")

# 👉 Start attendance:
# take_attendance()


In [22]:
capture_samples("student1")
capture_samples("student2")
label_ids = train_recognizer()
id_to_name = {v: k for k, v in label_ids.items()}


KeyboardInterrupt: 